# Программирование на Python.  
## Домашнее задание 5

реализовать класс Account, который моделирует поведение банковского счёта. Этот класс должен не только выполнять базовые операции,
но и вести детальный учёт всех действий, а также предоставлять аналитику по
истории операций.  
Этап 1. Реализация базового класса Account
Класс должен быть инициализирован с параметрами:
- account_holder (str): имя владельца счёта;
- balance (float, по умолчанию 0): начальный баланс счёта, не может быть
отрицательным.  
Атрибуты:
- holder: хранит имя владельца;
- _balance: приватный атрибут для хранения текущего баланса;
- operations_history: список или другая структура для хранения истории
операций.  
Важно: каждая операция должна храниться не просто как число, а как
структурированная информация, например, словарь или кортеж. Минимальный набор
данных для операции: тип операции ('deposit' или 'withdraw'), сумма, дата и время
операции, текущий баланс после операции, статус ('success' или 'fail').  
Этап 2. Реалирзация методов
1. __init__(self, account_holder, balance=0): конструктор;
2. deposit(self, amount): метод для пополнения счёта:
принимает сумму (должна быть положительной);
в случае успеха обновляет баланс и добавляет запись в историю операций.
3. withdraw(self, amount): метод для снятия средств:
- принимает сумму (должна быть положительной);
- проверяет, достаточно ли средств на счёте, если нет — операция не
проходит, но ее попытка с статусом 'fail' все равно фиксируется в истории;
- в случае успеха обновляет баланс и добавляет запись.
4. get_balance(self): метод, который возвращает текущий баланс.
5. get_history(self): метод, который возвращает историю операций.

In [12]:
import datetime

In [31]:
'''класс, который моделирует поведение банковского счёта.'''
class Account:
    def __init__(self, account_holder: str, balance: float = 0):
        if balance < 0:
            raise ValueError('Начальный баланс счёта, не может быть отрицательным')
        
        self.holder = account_holder
        self._balance = balance
        self.operations_history = []
    
    #Записываем начальный баланс
        if balance > 0:
            self.operations_history.append({
                'type': 'initial',
                'amount': balance,
                'time': datetime.datetime.now(),
                '_balance': balance,
                'status': 'success'
            })
    def deposit(self, amount: float):
        if amount <= 0:
            raise ValueError('Сумма должна быть положительной')        
        self._balance += amount
        operation = {
            'type': 'deposit',
            'amount': amount,
            'time': datetime.datetime.now(),
            '_balance': self._balance,
            'status': 'success'
        }
        self.operations_history.append(operation)
    
    def withdraw(self, amount: float):
        if amount <= 0:
            raise ValueError('Сумма должна быть положительной')
        if self._balance >= amount:
            self._balance -= amount
            self.operations_history.append({
                'type': 'withdraw',
                'amount': amount,
                'time': datetime.datetime.now(),
                '_balance': self._balance,
                'status': 'success'
            })
        else:
            self.operations_history.append({
                'type': 'withdraw',
                'amount': amount,
                'time': datetime.datetime.now(),
                '_balance': self._balance,
                'status': 'fail'
            })
            raise ValueError('Недостаточно средств')
    
    def get_balance(self):
        return self._balance
    
    def get_history(self):
        return self.operations_history.copy()

In [35]:
'''класс, который наследует всю функциональность класса Account и добавляет новую'''
class CreditAccount(Account):
    def __init__(self, holder: str, balance: float = 0, credit_limit: float = 0):
        if balance < -credit_limit:
            raise ValueError('Баланс ниже кредитного лимита')
        super().__init__(holder, balance)
        self.credit_limit = credit_limit
    
    def withdraw(self, amount: float):
        if amount <= 0:
            raise ValueError('Сумма должна быть больше нуля')
        if self._balance - amount >= -self.credit_limit:
            self._balance -= amount
            self.operations_history.append({
                'type': 'withdraw',
                'amount': amount,
                'time': datetime.datetime.now(),
                '_balance': self._balance,
                'status': 'success',
                'credit_used': self._balance < 0
            })
        else:
            self.operations_history.append({
                'type': 'withdraw',
                'amount': amount,
                'time': datetime.datetime.now(),
                '_balance': self._balance,
                'status': 'fail',
                'credit_used': False
            })
            raise ValueError('Превышен кредитный лимит')
    
    def get_available_credit(self):
        return self._balance + self.credit_limit

In [37]:
'''Пример использования'''
if __name__ == "__main__":
    # Тест дебитового счета
    acc = Account('Иван Иванов', 1000)
    acc.deposit(500)
    print(f"Баланс деб.: {acc.get_balance()}")
    
    # Тест кредитного счета
    credit_acc = CreditAccount('Петр Петров', 500, 2000)
    credit_acc.withdraw(1000)
    print(f"Баланс кр.: {credit_acc.get_balance()}")
    print(f"Доступный кредит: {credit_acc.get_available_credit()}")
    
    # История операций
    print('История операций:')
    for op in credit_acc.get_history():
        print(f"{op['type']}: {op['amount']} | Баланс: {op['_balance']} | Кредит: {op.get('credit_used', 'N/A')}")

Баланс деб.: 1500
Баланс кр.: -500
Доступный кредит: 1500
История операций:
initial: 500 | Баланс: 500 | Кредит: N/A
withdraw: 1000 | Баланс: -500 | Кредит: True
